# Training of a simple encoder transformer
- trained with a small balanced dataset 

In [ ]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)
    
MODEL_V = "04_simple_transformer"

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


## Splitting the data

In [13]:
from Dataset import Dataset

data_loader = Dataset(
    data_path="Fully_imputed_dataset.parquet",
    save_path="dataset_tensors.pth",
    method="oversample",
    balance=True,
    minority_ratio=0.3,
    target_column="SepsisLabel"
)

In [14]:
# as tensors
X_train, X_test, y_train, y_test = data_loader.get_train_test_tensors(size='small', sample_fraction=0.1)
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

Patient-level balance statistics:
Total patients: 53434
Label 0: 37404 patients (70.00%)
Label 1: 16030 patients (30.00%)
Imbalance ratio (majority/minority): 2.33
Negative-to-positive ratio (per patient): 2.33337492202121
Total records in reduced training set: 184542


## Transformer architecture

In [15]:
from architectures import TransformerClassifier


in_dim = X_train.shape[1]
model = TransformerClassifier(input_dim=in_dim)

In [16]:
from torch.utils.data import DataLoader, TensorDataset


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 512 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Training loop

In [26]:
from tqdm import tqdm
from torchmetrics import Accuracy, Precision, Recall
from torchvision.ops import sigmoid_focal_loss
from torch import nn


model = TransformerClassifier(input_dim=in_dim).to(device)
pos_weight = torch.tensor([2.3], dtype=torch.float32).to(device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001) 

epochs = 100
epoch_counter = []
loss_counter = []
acc_counter = []

t_accuracy = Accuracy(task='binary').to(device)
t_precision = Precision(task='binary').to(device)
t_recall = Recall(task='binary').to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss, epoch_acc, epoch_prec, epoch_rec = 0, 0, 0, 0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # forward pass
        y_logits = model(X_batch)
        # print("Logits sample:", y_logits[:5].detach().cpu().numpy())  # Should have positive values too
        y_probs = torch.sigmoid(y_logits)
        # because the dataset is too imbalance
        y_preds = (y_probs >= 0.3).float() 

        # loss function
        loss = loss_fn(y_logits, y_batch.unsqueeze(1).float())
        
        acc = t_accuracy(y_preds, y_batch.unsqueeze(1).float())
        prec = t_precision(y_preds, y_batch.unsqueeze(1).float())
        rec = t_recall(y_preds, y_batch.unsqueeze(1).float())
        
        # zero grad
        optimizer.zero_grad()
        # backpropagation
        loss.backward()
        # optimizer
        optimizer.step()

        # update epoch metrics
        epoch_loss += loss.item()
        epoch_acc += acc
        epoch_prec += prec
        epoch_rec += rec
        
        progress_bar.set_postfix({"Loss": loss.item(), "Acc": acc.item(), "Prec": prec.item(), "Rec": rec.item()})

    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)
    epoch_prec /= len(train_loader)
    epoch_rec /= len(train_loader)
    acc_counter.append(epoch_acc)
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.5f} | Accuracy: {epoch_acc:.2f}% | Precision: {epoch_prec:.2f}% | Recall: {epoch_rec:.2f}%")

Epoch 1/100:   0%|          | 0/361 [00:00<?, ?it/s]

Epoch 1/100:   0%|          | 0/361 [00:33<?, ?it/s, Loss=0.354, Acc=0.91, Prec=0.25, Rec=0.333]    

Epoch 1/100 | Loss: 0.43422 | Accuracy: 0.92% | Precision: 0.07% | Recall: 0.04%


Epoch 2/100 | Loss: 0.42555 | Accuracy: 0.91% | Precision: 0.15% | Recall: 0.07%


Epoch 3/100:   0%|          | 0/361 [00:30<?, ?it/s, Loss=0.325, Acc=0.941, Prec=0, Rec=0]          

...

Epoch 79/100:   0%|          | 0/361 [00:41<?, ?it/s, Loss=0.177, Acc=0.937, Prec=0.591, Rec=0.722] 

Epoch 79/100 | Loss: 0.11673 | Accuracy: 0.96% | Precision: 0.61% | Recall: 0.90%


Epoch 80/100 | Loss: 0.11827 | Accuracy: 0.95% | Precision: 0.61% | Recall: 0.90%


Epoch 81/100:   0%|          | 0/361 [00:26<?, ?it/s, Loss=0.0732, Acc=0.955, Prec=0.566, Rec=1]    

KeyboardInterrupt: 

## Save model and predictions

In [27]:
from pathlib import Path

# save the model
model_path = Path('./saved')
model_path.mkdir(exist_ok=True)
model_file = model_path / f"{MODEL_V}.pth"

torch.save(model.state_dict(), model_file)

In [28]:
model = TransformerClassifier(input_dim=in_dim)
model.load_state_dict(torch.load(f"./saved/{MODEL_V}.pth"))

<All keys matched successfully>